In [ ]:
import SimpleITK as sitk
import registration_utilities as ru
import registration_callbacks as rc

#%matplotlib inline

#%matplotlib widget
import matplotlib.pyplot as plt

from ipywidgets import interact, fixed

# utility method that either downloads data from the Girder repository or
# if already downloaded returns the file name for reading from disk (cached data)
%run update_path_to_download_script
from downloaddata import fetch_data as fdata

%matplotlib notebook
#%matplotlib inline
import gui
import registration_gui as rgui

import numpy as np
import os
OUTPUT_DIR = 'output'

In [ ]:
#Load image

fixed_image = sitk.ReadImage("Data/Pa3_CT.mha", sitk.sitkFloat64)
moving_image = sitk.ReadImage("Data/Pa3_MR.mha", sitk.sitkFloat64)

# Load mask

fixed_mask = sitk.ReadImage("Data/Pa3_CT_Mask.mha", sitk.sitkUInt8)
moving_mask = sitk.ReadImage("Data/Pa3_MR_Mask.mha", sitk.sitkUInt8)



In [ ]:
initial_transform = sitk.CenteredTransformInitializer(fixed_image, 
                                                      moving_image, 
                                                      sitk.Euler3DTransform(), 
                                                      sitk.CenteredTransformInitializerFilter.GEOMETRY)

In [ ]:
%%timeit -r1 -n1

global tx

registration_method = sitk.ImageRegistrationMethod()

# Similarity metric settings.
#registration_method.SetMetricAsMeanSquares()
#registration_method.SetMetricAsJointHistogramMutualInformation(numberOfHistogramBins=50)
registration_method.SetMetricAsMutualInformationEfficientEntropy(numberOfHistogramBins=50)
#registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
registration_method.SetMetricSamplingPercentage(0.01)

registration_method.SetInterpolator(sitk.sitkLinear)

# Optimizer settings.
registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=100, convergenceMinimumValue=1e-6, convergenceWindowSize=10)
#registration_method.SetOptimizerAsConjugateGradientLineSearch(learningRate=1.0, numberOfIterations=100, convergenceMinimumValue=1e-6, convergenceWindowSize=10)
#registration_method.SetOptimizerAsOnePlusOneEvolutionary(numberOfIterations=1000)
#registration_method.SetOptimizerAsPowell(numberOfIterations=1000)
#registration_method.SetOptimizerAsAmoeba(numberOfIterations=1000, simplexDelta=True)
#registration_method.SetOptimizerAsLBFGS2()
registration_method.SetOptimizerScalesFromPhysicalShift()


# Don't optimize in-place, we would possibly like to run this cell multiple times.
registration_method.SetInitialTransform(initial_transform, inPlace=False)

# Connect all of the observers so that we can perform plotting during registration.
registration_method.AddCommand(sitk.sitkStartEvent, rgui.start_plot)
registration_method.AddCommand(sitk.sitkEndEvent, rgui.end_plot)
registration_method.AddCommand(sitk.sitkMultiResolutionIterationEvent, rgui.update_multires_iterations) 
registration_method.AddCommand(sitk.sitkIterationEvent, lambda: rgui.plot_values(registration_method))

tx = registration_method.Execute(fixed_image, moving_image)

# Always check the reason optimization terminated.
print('Final metric value: {0}'.format(registration_method.GetMetricValue()))
print('Optimizer\'s stopping condition, {0}'.format(registration_method.GetOptimizerStopConditionDescription()))

In [ ]:
# Compute Dice and Hausdorff scores
# Transfer the segmentation via the estimated transformation. Use Nearest Neighbor interpolation to retain the labels.
moving_mask_pre_registration = sitk.Resample(
    moving_mask,
    fixed_mask,
    sitk.Transform(),
    sitk.sitkNearestNeighbor
)

moving_mask_post_registration = sitk.Resample(
    moving_mask,
    fixed_mask,
    tx,
    sitk.sitkNearestNeighbor
)

segmentation_label=100

label_overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
label_overlap_measures_filter.Execute(fixed_mask, moving_mask_pre_registration)
print(
    f"Dice coefficient before registration: {label_overlap_measures_filter.GetDiceCoefficient(segmentation_label):.2f}"
)

label_overlap_measures_filter.Execute(fixed_mask, moving_mask_post_registration)
print(
    f"Dice coefficient after registration: {label_overlap_measures_filter.GetDiceCoefficient(segmentation_label):.2f}"
)

label_overlap_measures_filter.Execute(fixed_mask, moving_mask_pre_registration)
print(
    f"Jaccard coefficient before registration: {label_overlap_measures_filter.GetJaccardCoefficient(segmentation_label):.2f}"
)

label_overlap_measures_filter.Execute(fixed_mask, moving_mask_post_registration)
print(
    f"Jaccard coefficient after registration: {label_overlap_measures_filter.GetJaccardCoefficient(segmentation_label):.2f}"
)

hausdorff_distance_image_filter = sitk.HausdorffDistanceImageFilter()
hausdorff_distance_image_filter.Execute(fixed_mask, moving_mask_pre_registration)
print(
    f"Hausdorff distance before registration: {hausdorff_distance_image_filter.GetHausdorffDistance():.2f}"
)
hausdorff_distance_image_filter.Execute(fixed_mask, moving_mask_post_registration)
print(
    f"Hausdorff distance after registration: {hausdorff_distance_image_filter.GetHausdorffDistance():.2f}"
)
